In [1]:
from config import dailyTemp
from datetime import timedelta, date
import fetch
from keplergl import KeplerGl
import pandas as pd

In [2]:
weatherStats = pd.read_json('../processed_data/temp_K_stats/F.json', orient='index')
weatherStats.sort_values(by=['var'], ascending=False, inplace=True)

weatherStats.head()

,std,var,avg,min,max
2015-10-19,17.685308,312.770124,47.392338,1.915522,70.855227
2015-02-20,15.848403,251.171887,-12.046808,-40.604616,14.382203
2015-01-31,15.442605,238.474034,5.673800,-31.844778,33.141524
2015-12-20,14.789917,218.741642,29.427599,-3.628370,50.821657
2015-04-09,14.531061,211.151741,55.817255,29.754000,78.098731


In [3]:
def getDayMap(m,d):
    df = fetch.get_weather(usecols='time,lat,lon,temp_K', datetime=date(2015,m,d))
    df['time'] = df['time'].apply(lambda x: pd.Timestamp(x[:10] + ' ' + x[11:]))
    df['temp_K'] = df['temp_K'].apply(lambda x: (x-273)*1.8 +32)
    df.rename(columns={'temp_K': 'temp_F'}, inplace=True)

    return KeplerGl(height=600,data={'day': df}, config=dailyTemp)

In [4]:
oct19 = getDayMap(10,19)

oct19

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


KeplerGl(config={'version': 'v1', 'config': {'visState': {'filters': [{'dataId': ['day'], 'id': 'fja631mn9', '…

In [5]:
# interesting
feb20 = getDayMap(2,20)

feb20

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


KeplerGl(config={'version': 'v1', 'config': {'visState': {'filters': [{'dataId': ['day'], 'id': 'fja631mn9', '…

In [6]:
jan31 = getDayMap(1,31)

jan31

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


KeplerGl(config={'version': 'v1', 'config': {'visState': {'filters': [{'dataId': ['day'], 'id': 'fja631mn9', '…

In [7]:
dec20 = getDayMap(12,20)

dec20

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


KeplerGl(config={'version': 'v1', 'config': {'visState': {'filters': [{'dataId': ['day'], 'id': 'fja631mn9', '…

In [8]:
# interesting
apr9 = getDayMap(4,9)

apr9

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


KeplerGl(config={'version': 'v1', 'config': {'visState': {'filters': [{'dataId': ['day'], 'id': 'fja631mn9', '…